# Convolutional Neural Network

### Importing the libraries

In [25]:
!pip install --upgrade tensorflow keras Pillow

Requirement already up-to-date: tensorflow in c:\users\nobre\anaconda3\lib\site-packages (2.4.1)
Requirement already up-to-date: keras in c:\users\nobre\anaconda3\lib\site-packages (2.4.3)
Requirement already up-to-date: Pillow in c:\users\nobre\anaconda3\lib\site-packages (8.2.0)


In [26]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [27]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [28]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [29]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [30]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [31]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [32]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [33]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Adding a third convolutional layer

In [34]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [35]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [36]:
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))

### Step 5 - Output Layer

In [37]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [38]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [39]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 34s 133ms/step - loss: 0.6900 - accuracy: 0.5354 - val_loss: 0.6060 - val_accuracy: 0.6900
Epoch 2/25
250/250 [==============================] - 32s 128ms/step - loss: 0.6182 - accuracy: 0.6591 - val_loss: 0.5572 - val_accuracy: 0.7100
Epoch 3/25
250/250 [==============================] - 33s 131ms/step - loss: 0.5801 - accuracy: 0.6955 - val_loss: 0.5246 - val_accuracy: 0.7465
Epoch 4/25
250/250 [==============================] - 33s 130ms/step - loss: 0.5369 - accuracy: 0.7262 - val_loss: 0.4934 - val_accuracy: 0.7620
Epoch 5/25
250/250 [==============================] - 32s 128ms/step - loss: 0.5093 - accuracy: 0.7537 - val_loss: 0.4698 - val_accuracy: 0.7795
Epoch 6/25
250/250 [==============================] - 32s 126ms/step - loss: 0.4819 - accuracy: 0.7619 - val_loss: 0.4748 - val_accuracy: 0.7735
Epoch 7/25
250/250 [==============================] - 33s 131ms/step - loss: 0.4489 - accuracy: 0.7938 - val_loss: 0.4491 - val_ac

## Part 4 - Making a single prediction

In [42]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [43]:
print(prediction)

dog


## Fim